Finite difference method for boundary value problem (Book 2, p. 10, ex. 1.5).

# Common parts

In [1]:
from numpy import exp, arange, array
from numpy.linalg import solve

import pandas as pd

We have to solve this task:

$-\frac{1}{x-3} u^{\prime \prime}+\left(1+\frac{x}{2}\right) u^{\prime}+e^{x / 2} u=2-x, u(-1)=u(1)=0$

In [2]:
def p(x):
    return 1/(x-3)

def q(x):
    return 1 + x/2

def r(x):
    return exp(x/2)

def f(x):
    return 2 - x

a = -1
b = 1
alpha = 0
beta = 0

# Differential equation approximation 

In [3]:
def prepare_coefficents(p, q, r, f, h, x):
    """Preparing equation coefficents.
    
    Suppose we given a boundary value problem 
    with bounds (a, b) and functions p, q, r, f. 
    We want to find values of  desired function 
    on n splits on (a, b). So we have step 
    of h=(b-a)/n. 
    Later we swap our differential formulas with
    their approximate values and grouping 
    coeficents near y_i values.
    
    Args:
        p, q, r, f (fun(x)): boundary value functions.
        h (float): step of points we want to find values in.
        x (float): point to calculate coefficents in.
    
    Returns:
        A, B, C, G (fun(x)): coefficents near y_i values.
    
    """
    
    A = -(p(x) / h**2) - (q(x) / 2*h)
    B = -(2*p(x) / h**2) - r(x)
    C = -(p(x) / h**2) + (q(x) / 2*h)
    G = f(x)
    
    return A, B, C, G

In [4]:
def solve_equation(p, q, r, f, a, b, alpha, beta, n):
    """Solve differential equation with boundaries.
    
    Args:
        p, q, r, f (fun(x)): boundary value functions.
        a, b (float): Bounds.
        alpha, beta (float): Bound values.
        n (int): Number of splits.
    
    Returns:
        Y (array-like<float>): Values of desired function
                              in (n + 1) points on (a, b)
                              with step h=(b-a)/n.
    """
    
    h = (b - a) / n
    points = arange(a, b + h, h)
    
    # ----- Generating matrix of coefficents and G vector. -----
    matrix = []
    G_vec = []
    
    # -- Filling first elements. --
    
    # Filling first row as [1, 0, 0, ...] because we
    # know exact value of y(a).
    first_row = []
    first_row.append(1)
    first_row += [0 for i in range(n)]
    matrix.append(first_row)
    
    # Filling first element in target array with alpha
    # because we know that y(a)=alpha.
    G_vec.append(alpha)
    
    A_s = []
    B_s = []
    C_s = []
    A_s.append(0)
    B_s.append(1)
    C_s.append(0)
    
    # -- Filling middle elements. --
    
    for i in range(1, n):
        A, B, C, G = prepare_coefficents(p, q, r, f, h, points[i])
        
        row = []
        row += [0 for j in range(i - 1)]
        row += [A, B, C]
        row += [0 for j in range(n + 1 - 3 - (i - 1))]
        
        matrix.append(row)
        G_vec.append(G)
        
        A_s.append(A)
        B_s.append(B)
        C_s.append(C)
        
    
    # -- Filling last elements. --
    
    # Filling last row as [0, 0, ..., 0, 1] because we
    # know exact value of y(b).
    last_row = []
    last_row += [0 for i in range(n)]
    last_row.append(1)
    matrix.append(last_row)
    
    # Filling last element in target array with beta
    # because we know that y(b)=beta.
    G_vec.append(beta)
    
    A_s.append(0)
    B_s.append(1)
    C_s.append(0)
    
    # ----- Solving equation. -----
    Y = solve(matrix, G_vec)
    
    
    return Y, points, [A_s, B_s, C_s, G_vec]
    

# Testing

In [5]:
# Number of splits.
n = 10

Y, x_s, stats = solve_equation(p, q, r, f, a, b, alpha, beta, n)

In [6]:
# Creating table.
data = pd.DataFrame(list(zip(x_s, stats[0], stats[1],
                             stats[2], stats[3], Y)), 
                    columns =['$x_i$', '$A_i$', '$B_i$', 
                              '$C_i$', '$G_i$', '$y_i$'])

data.set_index('$x_i$', inplace=True)
display(data)

,$A_i$,$B_i$,$C_i$,$G_i$,$y_i$
$x_i$,,,,,
-1.000000e+00,0.000000,1.000000,0.000000,0.0,-3.814695e-17
-8.000000e-01,6.518947,12.487575,6.638947,2.8,8.690983e-02
-6.000000e-01,6.874444,13.148071,7.014444,2.6,2.582800e-01
-4.000000e-01,7.272941,13.887152,7.432941,2.4,-1.986387e-01
-2.000000e-01,7.722500,14.720163,7.902500,2.2,4.412883e-01
-2.220446e-16,8.233333,15.666667,8.433333,2.0,-3.494906e-01
2.000000e-01,8.818571,16.751972,9.038571,1.8,4.555824e-01
4.000000e-01,9.495385,18.009366,9.735385,1.6,-3.042401e-01
6.000000e-01,10.286667,19.483475,10.546667,1.4,2.828076e-01
